In [1]:
%load_ext autoreload
%config Completer.use_jedi = False

In [5]:
%autoreload 2
import FEMOL
import numpy as np
np.set_printoptions(linewidth=200)
np.set_printoptions(suppress=True)
from IPython.display import clear_output
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, griddata, interp2d, RectBivariateSpline
import time
import runpy
import os
import sys

# Development Notebook for the FEMOL Project